建立User-URL二分图数据集

In [ ]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
 
from torch_geometric.data import HeteroData, download_url, extract_zip
from torch_geometric.transforms import ToUndirected, RandomLinkSplit

# 利用pandas查看数据集
print(pd.read_csv(movie_path).head())
print(pd.read_csv(rating_path).head())

In [ ]:
# 将电影名那列
# 利用嵌入模型将每个电影名用向量表示(Embedding)
class SequenceEncoder(object):
    # 初始化
    # 指定我们使用的嵌入模型
    # 和使用的设备
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        # 使用的设备
        self.device = device
        # 使用的嵌入模型名
        self.model = SentenceTransformer(model_name, device=device)
 
    # 嵌入模型不参与后续图神经网络的训练
    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(
            # 要进行嵌入的值
            df.values,
            # 显示处理进度
            show_progress_bar=True,
            # 转换为PyTorch的张量
            convert_to_tensor=True,
            # 使用的设备
            device=self.device
        )
        return x.cpu()

In [ ]:
# 将电影类型那列进行嵌入表示
class GenresEncoder(object):
 
    # 分隔符
    def __init__(self, sep='|'):
        self.sep = sep
 
    def __call__(self, df):
        # 分割出所有的电影类型
        # 后面两个for的逻辑是：
        # for col in df.values取出每一行的值
        # for g in col.split(self.sep)将取出来的值用指定的分隔符进行分割
        # set(g)将分割之后的结果转换为集合,去重
        genres = set(g for col in df.values for g in col.split(self.sep))
        # 将电影类型用数字表示
        mapping = {genre: i for i, genre in enumerate(genres)}
        # 用multi-hot形式表示电影的类型
        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for genre in col.split(self.sep):
                x[i, mapping[genre]] = 1
        return x

In [ ]:
# 从CSV文件中读取信息，建立二分图中节点的信息
def load_node_csv(path, index_col, encoders=None, **kwargs):
    """
    :param path: CSV文件路径
    :param index_col: 文件中的索引列，也就是节点所在的列
    :param encoders:节点嵌入器
    :param kwargs:
    :return:
    """
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    # 将索引用数字表示
    mapping = {index: i for i, index in enumerate(df.index.unique())}
    # 节点属性向量矩阵
    x = None
    # 如果嵌入器非空
    if encoders is not None:
        # 对相应的列进行嵌入
        # 获取嵌入向量表示
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)
 
    return x, mapping

In [ ]:
# 获取节点信息
# 处理movies.csv表，将'电影名','电影类型'列转换为嵌入向量的表示形式
movie_x, movie_mapping = load_node_csv(
    movie_path, index_col='movieId', encoders={
        # 电影名列的嵌入器
        'title': SequenceEncoder(),
        # 电影类型列的嵌入器
        'genres': GenresEncoder()
    })
# 处理ratings.csv表,将用户ID用PyTorch中的张量表示
user_x, user_mapping = load_node_csv(rating_path, index_col='userId')
# 建立异质图（这里具体是一个二分图）
# HeteroData()是PyG中内置的一个表示异质图的数据结构
data = HeteroData()
# 加入不同类型节点的信息
# 加入用户信息，用户没有属性向量
# 只需要告诉PyG有多少个用户节点就可以
data['user'].num_nodes = len(user_mapping)
# 告诉PyG 电影的属性向量矩阵，PyG会根据x推断出电影节点的个数
data['movie'].x = movie_x
print(data)
 

In [ ]:
# 建立用户和电影之间边的信息
# 将用户对电影的评分转换为PyTorch中的张量
# 方便后续模型的训练
class IdentityEncoder(object):
 
    def __init__(self, dtype=None):
        self.dtype = dtype
 
    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [ ]:
# 建立二分图边的连接信息
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    """
    :param path: CSV表的路径
    :param src_index_col: 二分图左边节点来源于CSV表的哪一列，比如'user_id'这列
    :param src_mapping:将user_id映射为节点编号，我们前面定义的user_mapping
    :param dst_index_col:同理，二分图右边电影节点
    :param dst_mapping:
    :param encoders:边的嵌入器
    :param kwargs:
    :return:
    """
    df = pd.read_csv(path, **kwargs)
    # 建立连接信息
    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    # 注意这里edge_index维度为[2,边数]
    edge_index = torch.tensor([src, dst])
    # 边的属性信息
    edge_attr = None
    # 如果嵌入器非空
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)
 
    return edge_index, edge_attr

In [ ]:
# 获取二分图边的信息
edge_index, edge_label = load_edge_csv(
    rating_path,
    # 二分图左边是用户
    src_index_col='userId',
    src_mapping=user_mapping,
    # 右边是电影
    dst_index_col='movieId',
    dst_mapping=movie_mapping,
    encoders={'rating': IdentityEncoder(dtype=torch.long)},
)
# 将二分图中的边命名为('user', 'rates', 'movie')
data['user', 'rates', 'movie'].edge_index = edge_index
data['user', 'rates', 'movie'].edge_label = edge_label
print(data)

In [ ]:
# 到此我们的异质图(这里是一个二分图)数据集就构建完毕了
# 下面进一步将其转换为一个真正可以进行训练的数据集
# 转换为无向图
data = ToUndirected()(data)
# 删除相反方向边的属性信息，因为没有电影对用户的评分数据
del data['movie', 'rev_rates', 'user'].edge_label
 
# 按照一定比例分割数据集为训练集、测试集、验证集
transform = RandomLinkSplit(
    num_val=0.05,
    num_test=0.1,
    # 负采样比率
    # 不用负采样，全部输入进行训练
    neg_sampling_ratio=0.0,
    # 告诉PyG边的连接关系
    edge_types=[('user', 'rates', 'movie')],
    rev_edge_types=[('movie', 'rev_rates', 'user')],
)
# 分割数据集
train_data, val_data, test_data = transform(data)
print(train_data)
print(val_data)
print(test_data)